In [ ]:
import PyPDF2
import pandas as pd

In [ ]:
pdf_file_path = "../amex/data/2025-02-27.pdf"

In [ ]:
# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page_num in range(len(reader.pages)):
            text += reader.pages[page_num].extract_text()
    return text

# Function to parse the extracted text
def parse_pdf_text(text):
    # This is just a placeholder and may need adjustment based on how transactions are structured in the PDF.
    # We'll assume each transaction is on a new line and has the format: "Date | Description | Amount"
    lines = text.split('\n')

    transactions = []
    for line in lines:
        parts = line.split()
        if len(parts) >= 3:  # Adjust depending on how your PDF is structured
            date = parts[0]
            description = ' '.join(parts[1:-1])
            amount = parts[-1]
            transactions.append([date, description, amount])

    # Convert to pandas DataFrame
    df = pd.DataFrame(transactions, columns=['Date', 'Description', 'Amount'])
    return df

In [ ]:
# Extract text from PDF
pdf_text = extract_text_from_pdf(pdf_file_path)

# Parse and display the transactions
transactions_df = parse_pdf_text(pdf_text)
transactions_df

In [ ]:
import pdfplumber
import pandas as pd
import re

def extract_amex_transactions(pdf_path):
    # Read PDF
    with pdfplumber.open(pdf_path) as pdf:
        text_pages = [page.extract_text() for page in pdf.pages]
    
    # Extract invoice period
    invoice_period_match = re.search(r"Fakturans period:\s*([\d.]+ till [\d.]+)", text_pages[0])
    invoice_period = invoice_period_match.group(1) if invoice_period_match else "Unknown"
    
    # Extract transaction sections
    ros_transactions_match = re.search(r"Nya köp förRoshan Talimi(.*?)Summa nya köp förRoshan Talimi", text_pages[1], re.DOTALL)
    steph_transactions_match = re.search(r"Nya köp förStephanie Maria GardnerExtrakort som slutar på31017(.*?)Summa nya köp förStephanie Maria Gardner", text_pages[1], re.DOTALL)
    
    ros_transactions_text = ros_transactions_match.group(1) if ros_transactions_match else ""
    steph_transactions_text = steph_transactions_match.group(1) if steph_transactions_match else ""
    
    # Function to extract transactions into DataFrame
    def extract_transactions(text):
        transactions = []
        pattern = r"(\d{2}\.\d{2}\.\d{2})\s+(\d{2}\.\d{2}\.\d{2})\s+(.+?)\s+(-?\d{1,3}(?:\.\d{3})*(?:,\d{2})?)"
        
        for match in re.finditer(pattern, text):
            transactions.append({
                "Transaction Date": match.group(1),
                "Process Date": match.group(2),
                "Description": match.group(3).strip(),
                "Amount (SEK)": float(match.group(4).replace('.', '').replace(',', '.'))
            })
        
        return pd.DataFrame(transactions)
    
    # Convert extracted transactions into DataFrames
    df_ros = extract_transactions(ros_transactions_text)
    df_steph = extract_transactions(steph_transactions_text)
    
    return invoice_period, df_ros, df_steph


In [ ]:
# Example usage
pdf_path = "../amex/data/original/2025-02-27.pdf"  # Path to your AmEx PDF invoice
output_dir = "../amex/data/output"  # Directory to save the extracted data
invoice_period, df_ros, df_steph = extract_amex_transactions(pdf_path)

print(f"Invoice Period: {invoice_period}")
print("\nRoshan Talimi Transactions:")
print(df_ros.head())
print("\nStephanie Maria Gardner Transactions:")
print(df_steph.head())

In [ ]:
df_ros